### Heroes Of Pymoli Data Analysis
* Only 14% of the players are women, which is a bit surprising given that women generally play RPGs at higher rates than that. (https://quanticfoundry.com/2017/01/19/female-gamers-by-genre/)
* The top spender in the game, Lisosia93, has only spent \\$18.96, which is also suprising, since one generally assumes that about 5\% of spenders in F2P games will be "whales", or players who spend over \\$100 over their lifetime in the game. (https://deltadna.com/blog/how-whales-spend/) This may be a new game, though.
* The average total purchase per person is highest in the 35-40 age range, which is not particularly surprising. But second place goes to the <10 age range. Perhaps this is because their parents (who are perhaps making the purchases) tend to be 35-40?
-----

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file)

In [2]:
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

In [3]:
pd.DataFrame(purchase_data['SN'].nunique(), index = [0], columns = ['Total Players'])

,Total Players
0,576


## Purchasing Analysis (Total)

In [4]:
purchase_data[['Item ID','Price']].drop_duplicates()['Price'].mean()

3.0433879781420767

In [38]:
unique_items = purchase_data['Item ID'].nunique()
avg_price = purchase_data['Price'].mean() # This is the mean price over all purchases
# To get the mean price over unique items you would do something like:
# purchase_data[['Item ID','Price']].drop_duplicates()['Price'].mean()
# which is slightly lower
num_purchases = purchase_data['Purchase ID'].count()
total_revenue = purchase_data['Price'].sum()

pd.DataFrame({'Number of Unique Items':unique_items,'Average Price':avg_price,
              'Number of Purchases':num_purchases,
              'Total Revenue':total_revenue}, index = [0])\
.style.format('${:,.2f}', subset = ['Average Price','Total Revenue'])

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


## Gender Demographics

In [10]:
purchase_data.pivot_table(index = 'Gender', values = 'SN', aggfunc = pd.Series.nunique)\
.sort_values('SN',ascending = False)\
.assign(perc_players = lambda x: 100*np.round(x['SN'] / x['SN'].sum(),4))\
.rename(columns = {'SN':'Total Count','perc_players':'Percentage of Players'})

,Total Count,Percentage of Players
Gender,,
Male,484,84.03
Female,81,14.06
Other / Non-Disclosed,11,1.91



## Purchasing Analysis (Gender)

In [28]:
pd.concat([
    purchase_data.pivot_table(index = 'Gender', values = 'Purchase ID', aggfunc = 'count')\
    .rename(columns = {'Purchase ID':'Purchase Count'}),
    purchase_data.pivot_table(index = 'Gender', values = 'Price', aggfunc = 'mean')\
    .rename(columns = {'Price':'Average Purchase Price'})\
    .applymap('${:,.2f}'.format),
    purchase_data.pivot_table(index = 'Gender', values = 'Price', aggfunc = 'sum')\
    .rename(columns = {'Price':'Total Purchase Value'})\
    .applymap('${:,.2f}'.format),
    purchase_data.groupby(['Gender','SN'], as_index=False)['Price'].sum().groupby(['Gender'])['Price'].mean()\
    .rename('Avg Total Purchase per Person')\
    .map('${:,.2f}'.format)
], axis = 1)

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

In [12]:
bins = [0, 9, 14, 19, 24, 29, 34, 39, 50]
purchase_data['age_bins'] = pd.cut(purchase_data['Age'], bins,
                                   labels = ['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+'])
purchase_data.groupby('age_bins')['SN'].nunique().to_frame('Total Count')\
.assign(perc_players = lambda x: 100*np.round(x['Total Count'] / x['Total Count'].sum(),4))\
.rename(columns={'perc_players':'Percentage of Players'}).rename_axis(None)

,Total Count,Percentage of Players
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


## Purchasing Analysis (Age)

In [29]:
pd.concat([
    purchase_data.pivot_table(index = 'age_bins', values = 'Purchase ID', aggfunc = 'count')\
    .rename(columns = {'Purchase ID':'Purchase Count'}),
    purchase_data.pivot_table(index = 'age_bins', values = 'Price', aggfunc = 'mean')\
    .rename(columns = {'Price':'Average Purchase Price'})\
    .applymap('${:,.2f}'.format),
    purchase_data.pivot_table(index = 'age_bins', values = 'Price', aggfunc = 'sum')\
    .rename(columns = {'Price':'Total Purchase Value'})\
    .applymap('${:,.2f}'.format),
    purchase_data.groupby(['age_bins','SN'], as_index=False)['Price'].sum().groupby(['age_bins'])['Price'].mean()\
    .rename('Avg Total Purchase per Person')\
    .map('${:,.2f}'.format)
], axis = 1).rename_axis(None)

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

In [30]:
top_five_total = purchase_data.groupby('SN')['Price'].sum().sort_values(ascending = False)\
.rename('Total Purchase Value')\
.head().map('${:,.2f}'.format)

In [31]:
pd.concat([purchase_data.groupby('SN')['Purchase ID'].count().loc[top_five_total.index].rename('Purchase Count'),
           purchase_data.groupby('SN')['Price'].mean().loc[top_five_total.index].rename('Average Purchase Price')\
           .map('${:,.2f}'.format),
           top_five_total], axis = 1)

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

In [32]:
item_details = purchase_data[['Item ID','Item Name','Price']].groupby(['Item ID','Item Name'])\
.agg(['count','max','sum'])\
.sort_values([('Price','count')], ascending = False)

item_details.columns = ['Purchase Count','Item Price','Total Purchase Value']

item_details_formatted = item_details.copy()
item_details_formatted['Item Price'] = item_details_formatted['Item Price'].map('${:,.2f}'.format)
item_details_formatted['Total Purchase Value'] = item_details_formatted['Total Purchase Value'].map('${:,.2f}'.format)
item_details_formatted.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

In [33]:
item_details_sorted = item_details.sort_values('Total Purchase Value', ascending = False)

item_details_sorted['Item Price'] = item_details_sorted['Item Price'].map('${:,.2f}'.format)
item_details_sorted['Total Purchase Value'] = item_details_sorted['Total Purchase Value'].map('${:,.2f}'.format)
item_details_sorted.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
